# Custom GNN module
Sometimes, your model goes beyond simply stacking existing GNN modules. For example, you would like to invent a new way of aggregating neighbor information by considering node importance or edge weights.

Goals:

- Understand DGL’s message passing APIs.

- Implement GraphSAGE convolution module by your own.

In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

Using backend: pytorch


## Message passing and GNNs
DGL follows the message passing paradigm inspired by the Message Passing Neural Network proposed by [Gilmer et al.](https://arxiv.org/abs/1704.01212) Essentially, they found many GNN models can fit into the following framework:

$m^{(l)}_{u→v}=M^{(l)} \left( h^{(l-1)}_v,h^{(l-1)}_u,e^{(l-1)}_{u→v} \right)$

$m^{(l)}_v = \sum_{u \in N(v)}m^{(l)}_{u→v}$

$h_v^{(l)} = U^{(l)} \left( h_v^{(l-1)}, m_v^{(l)}\right)$

Where $M^{(l)}$ is the **message function**, $\sum$ the **reduce function** (does not have to be a summation) and $U^{(l)}$ the **update function**.

Example [GraphSAGE convolution (Hamilton et al., 2017)](https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf) :

$h^k_{N(v)} \leftarrow \mathrm{Average} \{h_u^{k-1}, \forall u \in N(v) \}$

$h^k_{v} \leftarrow \mathrm{ReLU} \{W^k \cdot \mathrm{CONCAT}(h_v^{k-1}, h_{N(v)}^k)\}$

Note: message passing is directional: message from node $u$ to $v$ is not necessarily the same as $v$ to $u$.

In [2]:
import dgl.function as fn

class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # a linear submodule for projecting the input and neighbor feature to the output
        self.linear = nn.Linear(in_feat * 2, out_feat)
        
    def forward(self, g, h):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        with g.local_scope():
            g.ndata['h'] = h
            # update_all is a message passing API.
            g.update_all(message_func=fn.copy_u('h', 'm'), reduce_func=fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

The central piece in this code is the `g.update_all` function, which gathers and averages the neighbor features. There are three concepts here:

Message function `fn.copy_u('h', 'm')` that copies the node feature under name `'h'` as messages sent to neighbors.

Reduce function `fn.mean('m', 'h_N')` that averages all the received messages under name `'m'` and saves the result as a new node feature `'h_N'`.

`update_all` tells DGL to trigger the message and reduce functions for all the nodes and edges.

Now we are building a multilayer GraphSAGE network.

In [3]:
class GraphSAGENet(nn.Module):
    def __init__(self, in_feat, hidden_feat, num_classes):
        super(GraphSAGENet, self).__init__()
        self.conv1 = SAGEConv(in_feat, hidden_feat)
        self.conv2 = SAGEConv(hidden_feat, num_classes)
        
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [4]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0] # dataset has only one graph

def train(g, model, epochs: int):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0
    
    print(g.ndata.keys())
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    
    for epoch in range(epochs):
        # Forward
        logits = model(g, features)
        
        # Compute prediction
        pred = logits.argmax(1)
        
        # Compute loss (only on nodes of the training set!)
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])
        
        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()
        
        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
            
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())
        
        if epoch % 5 == 0:
            print(f'Epoch: {epoch}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})')
            
#g = g.to('cuda')
model = GraphSAGENet(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model, epochs=200)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
dict_keys(['feat', 'label', 'test_mask', 'val_mask', 'train_mask'])
Epoch: 0, loss: 1.948, val acc: 0.114 (best 0.114), test acc: 0.103 (best 0.103)
Epoch: 5, loss: 1.878, val acc: 0.326 (best 0.326), test acc: 0.297 (best 0.297)
Epoch: 10, loss: 1.731, val acc: 0.338 (best 0.340), test acc: 0.351 (best 0.325)
Epoch: 15, loss: 1.507, val acc: 0.416 (best 0.416), test acc: 0.405 (best 0.405)
Epoch: 20, loss: 1.218, val acc: 0.500 (best 0.500), test acc: 0.484 (best 0.484)
Epoch: 25, loss: 0.900, val acc: 0.628 (best 0.628), test acc: 0.581 (best 0.581)
Epoch: 30, loss: 0.604, val acc: 0.700 (best 0.700), test acc: 0.678 (best 0.678)
Epoch: 35, loss: 0.372, val acc: 0.758 (best 0.758), test acc: 0.732 (best 0.732)
Epoch: 40, loss: 0.218, val acc: 0.748 (best 0.760), test acc: 0.760 (best 0.738)
Epoch: 45, los

Incorporating edge features like weights into a SAGEConv layer

In [5]:
class WeightedSAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model with edge weights.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    def __init__(self, in_feat, out_feat):
        super(WeightedSAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h, w):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        w : Tensor
            The edge weight.
        """
        with g.local_scope():
            g.ndata['h'] = h
            g.edata['w'] = w
            g.update_all(message_func=fn.u_mul_e('h', 'w', 'm'), reduce_func=fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

Because the graph in this dataset does not have edge weights, we manually assign all edge weights to one in the `forward()` function of the model. 

In [6]:
class WeightedGrapSAGENet(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(WeightedGrapSAGENet, self).__init__()
        self.conv1 = WeightedSAGEConv(in_feats, h_feats)
        self.conv2 = WeightedSAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat, torch.ones(g.num_edges()).to(g.device))
        h = F.relu(h)
        h = self.conv2(g, h, torch.ones(g.num_edges()).to(g.device))
        return h

model = WeightedGrapSAGENet(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model, 200)

dict_keys(['feat', 'label', 'test_mask', 'val_mask', 'train_mask'])
Epoch: 0, loss: 1.951, val acc: 0.316 (best 0.316), test acc: 0.319 (best 0.319)
Epoch: 5, loss: 1.884, val acc: 0.354 (best 0.360), test acc: 0.388 (best 0.372)
Epoch: 10, loss: 1.760, val acc: 0.468 (best 0.570), test acc: 0.462 (best 0.553)
Epoch: 15, loss: 1.571, val acc: 0.482 (best 0.570), test acc: 0.481 (best 0.553)
Epoch: 20, loss: 1.316, val acc: 0.596 (best 0.596), test acc: 0.565 (best 0.565)
Epoch: 25, loss: 1.016, val acc: 0.658 (best 0.658), test acc: 0.652 (best 0.652)
Epoch: 30, loss: 0.717, val acc: 0.700 (best 0.700), test acc: 0.705 (best 0.705)
Epoch: 35, loss: 0.466, val acc: 0.724 (best 0.724), test acc: 0.731 (best 0.731)
Epoch: 40, loss: 0.288, val acc: 0.736 (best 0.742), test acc: 0.747 (best 0.740)
Epoch: 45, loss: 0.175, val acc: 0.736 (best 0.742), test acc: 0.756 (best 0.740)
Epoch: 50, loss: 0.108, val acc: 0.738 (best 0.742), test acc: 0.764 (best 0.740)
Epoch: 55, loss: 0.069, val acc:

### Custom message and reduce functions examples

In [7]:
def custom_u_mul_e(edges): # edges has 3 members: src, dst (source/destination node), data (edge feature)
    return {'m': edges.src['h'] * edges.data['w']}

def custom_sum(nodes):
    return {'h': nodes.mailbox['m'].sum(1)}

In [8]:
print(g.ndata['feat'])

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [12]:
print(g.number_of_nodes())
print(len(g.ndata['feat'][0]))

2708
1433
